In [2]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
from pulp import *

In [3]:
query = '''

    SELECT 
        first_name
        ,second_name
        ,web_name
        ,t1.id_
        ,now_cost
        ,points_per_game
        ,selected_by_percent
        ,team
        ,team_code
        ,total_points
        ,minutes
        ,goals_scored
        ,assists
        ,clean_sheets
        ,goals_conceded
        ,yellow_cards
        ,red_cards
        ,bonus
        ,cost_change_start
        ,cost_change_event
        ,dreamteam_count
        ,selected_by_percent
        ,t2.plural_name_short as player_position
        ,squad_select
        ,squad_min_play
        ,squad_max_play
        ,cast(cast(total_points as decimal(20,2))/cast(now_cost as decimal(20,2)) as decimal(10,2)) 
            AS points_per_cost
        ,date_inserted
 
    FROM fantasy_football t1

    INNER JOIN player_types t2
        ON t1.element_type = t2.id_

    WHERE   
        

'''

In [4]:
def execute_query(query):
    conn = psycopg2.connect("dbname=fantasy_football user=postgres password=ca*m<>bL45g")
    cur = conn.cursor()
    cur.execute(query)
    column_names = [col_name[0] for col_name in cur.description]
    data = cur.fetchall()
    cur.close()
    conn.close()
    df = pd.DataFrame(data)
    df.columns = column_names
    return df

In [5]:
execute_query(query)

,first_name,second_name,web_name,id_,now_cost,points_per_game,selected_by_percent,team,team_code,total_points,...,bonus,cost_change_start,cost_change_event,dreamteam_count,selected_by_percent,player_position,squad_select,squad_min_play,squad_max_play,points_per_cost
0,Cédric,Alves Soares,Cédric,1,42,0.0,0.2,1,3,0,...,0,-3,0,0,0.2,DEF,5,3,5,0.00
1,Cédric,Alves Soares,Cédric,1,42,0.0,0.2,1,3,0,...,0,-3,0,0,0.2,DEF,5,3,5,0.00
2,Granit,Xhaka,Xhaka,3,50,4.3,2.9,1,3,26,...,2,0,0,1,2.9,MID,5,2,5,0.52
3,Granit,Xhaka,Xhaka,3,50,4.3,2.9,1,3,26,...,2,0,0,1,2.9,MID,5,2,5,0.52
4,Mohamed,Elneny,Elneny,4,43,2.0,0.8,1,3,2,...,0,-2,0,1,0.8,MID,5,2,5,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,Matheus Luiz,Nunes,Matheus,589,50,4.0,0.3,20,39,16,...,3,0,0,0,0.3,MID,5,2,5,0.32
1252,Sasa,Kalajdzic,Kalajdžić,608,55,1.0,0.0,20,39,1,...,0,0,0,0,0.0,FWD,3,1,3,0.02
1253,Sasa,Kalajdzic,Kalajdžić,608,55,1.0,0.0,20,39,1,...,0,0,0,0,0.0,FWD,3,1,3,0.02
1254,Diego,Da Silva Costa,Diego Costa,625,55,0.0,0.2,20,39,0,...,0,0,0,0,0.2,FWD,3,1,3,0.00


In [6]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
from pulp import *

class BestPlayers_test:
    def __init__(self, df):
        self.df = df
        self.prob= self.create_problem_variable()
        self.decision_variables = []
        self.optimisation_result = self.solve_optimisation_problem()
    

    def create_problem_variable(self):
        self.prob = pulp.LpProblem('FantasyTeam', LpMaximize)
        

    def create_decision_variables(self):
        #self.decision_variables = []
        for rownum, row in self.df.iterrows():
            variable = str('x' + str(rownum))
            variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
            self.decision_variables.append(variable)
    
    def create_constraints(self, constraint_column, constraint_name, constraint, sign):
        ''' 
        
        Function to create constraints for the constrained optimisation.

        The sign argument can take value '=', '<=', '>=', '>', '<'
        
        
        '''
        constraint_string = ""
        for row_idx, row in self.df.iterrows():
            for idx, player in enumerate(self.decision_variables):
                if row_idx == idx:
                    if constraint_name in ['GKP', 'DEF', 'MID', 'FWD']:
                        if row[constraint_column] == constraint_name:
                            constraint_string += 1*player
                    else :
                        constraint_string += row[constraint_column]*player

        if sign == '=':
            self.prob += (constraint_string == constraint)
        elif sign == '<=':
            self.prob += (constraint_string <= constraint)
        elif sign == '>=':
            self.prob += (constraint_string >= constraint)
        elif sign == '<':
            self.prob += (constraint_string < constraint)
        elif sign == '>':
            self.prob += (constraint_string > constraint)

    def create_objective_function(self):
        total_points = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    formula = row['total_points']*player
                    total_points += formula

        self.prob += total_points

    def create_team_constraints(self):
        team_dict= {}
        for team in set(self.df.team):
            team_dict[str(team)]=dict()
            team_dict[str(team)]['avail'] = 3
            team_dict[str(team)]['total'] = ""
            for rownum, row in self.df.iterrows():
                for i, player in enumerate(self.decision_variables):
                    if rownum == i:
                        if row['team'] == team:
                            formula = 1*player
                            team_dict[str(team)]['total'] += formula

            self.prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])


    def create_optimisation_problem(self):
        self.create_problem_variable()
        self.create_decision_variables()
        self.create_objective_function()
        constraints = ['GKP', 'DEF', 'MID', 'FWD']
        players_allowed = [2,5,5,3]
        for i, j in zip(constraints, players_allowed):
            self.create_constraints('player_position', i, j, '=')
        self.create_constraints('now_cost', 'cost', 1000, '<=')
        self.create_team_constraints()


    def solve_optimisation_problem(self):
        self.create_optimisation_problem()
        self.prob.writeLP('FantasyTeam.lp')
        optimization_result = self.prob.solve()
        self.optimisation_result = optimization_result 


    def return_team(self):
        self.solve_optimisation_problem()
        variable_name = []
        variable_value = []

        for v in self.prob.variables():
            variable_name.append(v.name)
            variable_value.append(v.varValue)

        data = pd.DataFrame({'variable': variable_name, 'value': variable_value})
        for rownum, row in data.iterrows():
            value = re.findall(r'(\d+)', row['variable'])
            data.loc[rownum, 'variable'] = int(value[0])

        data = data.sort_index()

        #append results
        for rownum, row in self.df.iterrows():
            for results_rownum, results_row in data.iterrows():
                if rownum == results_row['variable']:
                    self.df.loc[rownum, 'decision'] = results_row['value']

        self.df[self.df.decision==1].now_cost.sum() 
        self.df[self.df.decision==1].total_points.sum() 
        best_squad = self.df[self.df.decision==1].sort_values('player_position').head(15)

        ####check constraints hold:
        if max(list(best_squad['team'].value_counts()))>3:
            print('Team Constraint Failed')
        else:
            print('Maximum number of players from one team is 3 or less')

        if best_squad['now_cost']> 1000:
            print('Cost constraint failed')
        else:
            print('Cost constraint holds')
        
        return best_squad


    

            

In [7]:
test = BestPlayers_test(execute_query(query)).return_team()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/fraserbrown/fantasy_football/.venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/7713067306f84f7fa3cefaf3d8e2bbed-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/7713067306f84f7fa3cefaf3d8e2bbed-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 7113 RHS
At line 7139 BOUNDS
At line 8396 ENDATA
Problem MODEL has 25 rows, 1256 columns and 3768 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 604 - 0.00 seconds
Cgl0003I 0 fixed, 5 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 550 columns (550 integer (0 of which binary

In [20]:
max(list(test['team'].value_counts()))

3

In [40]:
test['now_cost'].sum()

996

In [1]:
import os
PASSWORD = os.environ['POSTGRES_DB_PASSWORD']

In [3]:
PASSWORD

'ca*m<>bL45g'

In [2]:
print(PASSWORD)

ca*m<>bL45g


In [63]:
print (os.environ)

environ({'ELECTRON_RUN_AS_NODE': '1', 'SECURITYSESSIONID': '186a3', 'USER': 'fraserbrown', 'MallocNanoZone': '0', '__CFBundleIdentifier': 'com.microsoft.VSCode', 'COMMAND_MODE': 'unix2003', 'PATH': '/Users/fraserbrown/fantasy_football/.venv/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin', 'SHELL': '/bin/bash', 'HOME': '/Users/fraserbrown', '__CF_USER_TEXT_ENCODING': '0x1F5:0:2', 'LaunchInstanceID': '2320BCA8-983B-4E42-AF93-FB82654DD87B', 'XPC_SERVICE_NAME': '0', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.UILniHqVTA/Listeners', 'XPC_FLAGS': '0x0', 'LOGNAME': 'fraserbrown', 'TMPDIR': '/var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'VSCODE_CWD': '/', 'VSCODE_NLS_CONFIG': '{"locale":"en-gb","availableLanguages":{},"_languagePackSupport":true}', 'VSCODE_CODE_CACHE_PATH': '/Users/fraserbrown/Library/Application Support/Code/CachedData/784b0177c56c607789f9638da7b6bf3230d47a8c', 'VSCODE_IPC_HOOK': '/Users/fraserbrown/Library/Application